In [1]:
#import packages
#pip install textblob
#pip install keras
#pip install tensorflow
import time
from textblob import TextBlob
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras.optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

2023-04-08 19:27:23.836192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#df_09= pd.read_pickle(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\2009_preprocessed_date.pickle") 
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/2009_text_wo_names.pickle")

In [3]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

#PP Data
df_09['col_type'] = df_09.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)
df_09= df_09[df_09["col_type"].notnull()]
df_09

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
9,"[leave, band, follow, bust, say, simply, could...",3,0,"[Noel, Gallagher]",True,Noel Gallagher left the Manchester band follow...,5048,2009,0.0
10,"[launch, clothing, line, earlier, year, admit,...",2,0,[Liam],None,"""Liam launched his clothing line Pretty Green ...",5048,2009,0.0
11,"[thinking, next, step, musically, mind, say]",1,0,[],None,"""I'm thinking of what the next step is musical...",5048,2009,0.0
13,"[people, able, buy, record]",1,0,[],None,"""People will be able to buy his records.",5048,2009,0.0
33,"[come, back, hang, fellow, cast, member, mitch...",0,2,[Sam],None,She's coming back as Sam Mitchell and was hang...,8981,2009,1.0
...,...,...,...,...,...,...,...,...,...
19327,"[report, speculate, mime, part, track, want, p...",0,2,[],None,Some reports have speculated that she mimed pa...,2157826,2009,1.0
19328,"[know, dance, lot]",0,1,[],None,"""I know she was dancing a lot.",2157826,2009,1.0
19329,"[know, mime, think, really, great, performance...",0,2,[],None,I don't know if she was miming or not but I th...,2157826,2009,1.0
19330,"[legend, make, comeback, show, rendition, new,...",2,0,"[Pop, Robbie]",True,Pop legend Robbie Williams made his comeback o...,2157826,2009,0.0


In [4]:
def subjectivity(sentence):
    subjectivity = ""

    subjectivity = TextBlob(sentence).sentiment.subjectivity

    return subjectivity

def polarity(sentence):
    polarity = ""
    polarity = TextBlob(sentence).sentiment.polarity
    return polarity

In [5]:
df_09['subjectivity'] = df_09['sentences'].apply(subjectivity)
#df_09['polarity'] = df_09['sentences'].apply(polarity)
df_09

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type,subjectivity
9,"[leave, band, follow, bust, say, simply, could...",3,0,"[Noel, Gallagher]",True,Noel Gallagher left the Manchester band follow...,5048,2009,0.0,0.152381
10,"[launch, clothing, line, earlier, year, admit,...",2,0,[Liam],None,"""Liam launched his clothing line Pretty Green ...",5048,2009,0.0,0.600000
11,"[thinking, next, step, musically, mind, say]",1,0,[],None,"""I'm thinking of what the next step is musical...",5048,2009,0.0,0.000000
13,"[people, able, buy, record]",1,0,[],None,"""People will be able to buy his records.",5048,2009,0.0,0.625000
33,"[come, back, hang, fellow, cast, member, mitch...",0,2,[Sam],None,She's coming back as Sam Mitchell and was hang...,8981,2009,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
19327,"[report, speculate, mime, part, track, want, p...",0,2,[],None,Some reports have speculated that she mimed pa...,2157826,2009,1.0,0.000000
19328,"[know, dance, lot]",0,1,[],None,"""I know she was dancing a lot.",2157826,2009,1.0,0.000000
19329,"[know, mime, think, really, great, performance...",0,2,[],None,I don't know if she was miming or not but I th...,2157826,2009,1.0,0.750000
19330,"[legend, make, comeback, show, rendition, new,...",2,0,"[Pop, Robbie]",True,Pop legend Robbie Williams made his comeback o...,2157826,2009,0.0,0.264610


In [ ]:
df_09["subjectivity"].describe()

In [6]:
#create a 2 point criteria from 0 to 1 (range of polarity)
def map_sentiment(value):
    if value >= 0.5: 
        return 1
    #elif value <= -0.33:
        #return -1
    else:
        return 0


df_09['sentiment'] = df_09['subjectivity'].apply(map_sentiment)
df_09['sentiment'] = df_09["sentiment"].astype(float)
df_09["sentiment"].value_counts()

0.0    3474
1.0    1868
Name: sentiment, dtype: int64

**RNN LSTM Model for Sentiment Analysis** 

This classifier identifies the polarity of the sentences based on the subjectivity. It will be used to cross-reference sentiment along with the male or female nature of the columns. 

In [ ]:
#BEST FUNCTION
def rnn_lstm(df, sentences_col, sentiment_col, gender_col):
    # Start timer 
    start_time = time.time()

    X = df[sentences_col]
    y = df[sentiment_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #gender col 
    gender_col = df[gender_col]
    
    # Tokenize the data
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(X_train)
    
    # Convert the texts to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Pad the sequences to ensure equal length
    maxlen = 100
    X_train_padded = pad_sequences(X_train_seq, padding='post', maxlen=maxlen)
    X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=maxlen)
    
    # Define the LSTM model
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=1000, output_dim=64, input_length=maxlen))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy', "mean_squared_error"])
    
    # Train the model
    model_lstm.fit(X_train_padded, y_train, epochs=3, batch_size=32, validation_data=(X_test_padded, y_test))
    
    # Evaluate the model
    score = model_lstm.evaluate(X_test_padded, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Add early stopping 
    earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

    # Make predictions on the padded sequences
    y_pred = model_lstm.predict(X_test_padded)
    
    # Convert sequences back to strings for TextBlob analysis
    X_train_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_train_seq]
    X_test_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_test_seq]
    
    # Calculate the sentiment polarity of each sentence using TextBlob
    X_train_polarity = [TextBlob(sent).sentiment.polarity for sent in X_train_str]
    X_test_polarity = [TextBlob(sent).sentiment.polarity for sent in X_test_str]
    
    # Convert polarity values to sentiment labels
    y_train_sentiment = ['positive' if polarity > 0.33 else 'negative' if polarity <- 0.33 else 'neutral' for polarity in X_train_polarity]
    y_test_sentiment = ['positive' if polarity > 0.33 else'negative' if polarity <- 0.33 else 'neutral' for polarity in X_test_polarity]
                        

    # Create a DataFrame of the test data, predicted sentiment, and gender
    df_results = pd.DataFrame({'Sentence': X_test, 'Sentiment': y_test_sentiment, 'Gender': gender_col[X_test.index]}) #,'Predicted Sentiment': y_test_sentiment.flatten()})
    
    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_results


In [ ]:
senti_df09 = rnn_lstm(df_09, "sentences", "sentiment", "col_type")
senti_df09

**CNN Model for Sentiment Analysis (WE WILL NOT USE, SIMPLY FOR REFERENCE TO ANSWER DEFENSEN QUESTIONS)**

In [7]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [ ]:
#virgin code CNN
# Define X and y
X = df_09['sentences']
y = df_09['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the CNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

X_test = pd.DataFrame(X_test)

# Get the corresponding gender value
gender = df_09.iloc[X_test.index]['col_type']
#gender = pd.DataFrame(gender)
#all_df = pd.concat([X_test, y_test, gender], axis = 1)

print(gender, y_pred)

In [8]:
def cnn(df):
    #virgin code CNN
    X = df['sentences']
    y = df['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
    maxlen = 100
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the CNN model
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])

# Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

    y_pred = model.predict(X_test)

# Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print(type(X_test))

    X_test = tokenizer.sequences_to_texts(X_test)


# Get the corresponding gender value
    X_t = pd.DataFrame(X_test)
    gender = df.iloc[X_t.index]['col_type']
    print(type(gender))
    y_p = pd.DataFrame(y_pred, columns = ["Senti"])
    g = pd.DataFrame(gender)

    return X_t, y_p, g
    #return all


In [9]:
cnn_result = cnn(df_09)

Epoch 1/3
67/67 [==============================] - 2s 18ms/step - loss: 0.6311 - accuracy: 0.6412 - mean_squared_error: 0.2201 - val_loss: 0.5830 - val_accuracy: 0.6726 - val_mean_squared_error: 0.1974
Epoch 2/3
67/67 [==============================] - 1s 16ms/step - loss: 0.4196 - accuracy: 0.8189 - mean_squared_error: 0.1295 - val_loss: 0.3355 - val_accuracy: 0.8606 - val_mean_squared_error: 0.1024
Epoch 3/3
34/34 [==============================] - 0s 3ms/step
Test loss: 0.21873831748962402
Test accuracy: 0.9232928156852722
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [11]:
df0 = cnn_result[0]
df1 = cnn_result[1]
df2 = cnn_result[2]

In [ ]:
df0

In [ ]:
df1

In [16]:
df2

,col_type
9,0.0
10,0.0
11,0.0
13,0.0
33,1.0
...,...
3830,0.0
3834,0.0
3840,0.0
3841,0.0


In [ ]:

#from keras.models import Sequential
#from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
#from keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
#from sklearn.model_selection import train_test_split

# Define X and y
X = df_09['sentences']
y = df_09['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the CNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert predictions to sentiment labels
y_test_sentiment = ['positive' if polarity > 0.33 else 'negative' if polarity <- 0.33 else 'neutral' for polarity in X_test]
#y_test_sentiment = np.where(y_pred > 0.5, 'Positive', 'Negative')

# Create DataFrame of results
df_results = pd.DataFrame({'Sentence': X_test, 'Sentiment': y_test_sentiment})

# Print the DataFrame
print(df_results)


# End timer 
end_time = time.time()
print(f"\nExecution time: {end_time - start_time:.2f} seconds")

#return df_results


In [ ]:
def get_gender_df(X_test, df, gender_col):
    gender = df[gender_col].values[X_test.index]
    gender_df = pd.DataFrame({'Sentence': X_test.flatten(), 'Gender': gender.flatten()})
    return gender_df

In [ ]:
# Call the function to get the gender DataFrame
gender_df = get_gender_df(X_test, df_09, 'col_type')

# Combine the gender DataFrame with the results DataFrame
df_results = pd.concat([df_results, gender_df], axis=1)


**Table Comparing Model Results**

In [ ]:
results = {' ': ['LSTM', 'CNN'],
           'Accuracy': [0.795, 0.866],
           'Mean Squared Error': [0.193, 0.1339],
           'Test loss': [0.3425, -334.315]}

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(results)

# Set the index of the dataframe to the Kernel column
df.set_index(' ', inplace=True)

# Display the dataframe
print(df)